In [296]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

os.chdir(r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\22 July 2025\2003")
os.getcwd()

# Filepaths for the datasets
filepaths = {
    'df1': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 3_Household characteristics.dta",
    'df2': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 13_financial assets other than shares & debentures owned by the household.dta",
    'df3': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 12_shares & debentures owned by the household in co operative societies & companies.dta",
    'df4': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS2~1\VIB918~1.DTA",
    'df5': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS2~1\VI4642~1.DTA",
    'df6': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 5_land owned by the household.dta",
    'df7': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 6 part 1_buildings and other constructions owned by the household.dta",
    'df8': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2003 nss 59th round\AIDIS 2003 All Datasets\Visit 1 & 2 Combined_Block 6 part 2_buildings and other constructions owned by the household.dta",
}

In [297]:
# Load dataframes
dataframes = {name: pd.read_stata(path) for name, path in filepaths.items()}

# Print unique HHIDs in first seven datasets
for i in range(1, 8):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs across the first five datasets
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 9)
]) #& set.union(*[
    #set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 8)
#])
print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 9):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6, df7, df8 = [dataframes[f'df{i}'] for i in range(1, 9)]


Unique HHIDs in df1: 139041
Unique HHIDs in df2: 124906
Unique HHIDs in df3: 9661
Unique HHIDs in df4: 144714
Unique HHIDs in df5: 144716
Unique HHIDs in df6: 118622
Unique HHIDs in df7: 118674
Common HHIDs across: 6342
df1: 6342 rows after filtering
df2: 21266 rows after filtering
df3: 13027 rows after filtering
df4: 10549 rows after filtering
df5: 10549 rows after filtering
df6: 21428 rows after filtering
df7: 15972 rows after filtering
df8: 15884 rows after filtering


In [298]:
df1.to_stata("df1_2003.dta", write_index=False)
df2.to_stata("df2_2003.dta", write_index=False)
df3.to_stata("df3_2003.dta", write_index=False)
df4.to_stata("df4_2003.dta", write_index=False)
df5.to_stata("df5_2003.dta", write_index=False)
df6.to_stata("df6_2003.dta", write_index=False)
df7.to_stata("df7_2003.dta", write_index=False)
df8.to_stata("df8_2003.dta", write_index=False)

In [299]:
df1.columns

Index(['HHID', 'CentreCodeRndShift', 'Vill_Blk_Slno', 'Round', 'Schedule',
       'Sample', 'Sector', 'State_Region', 'State', 'District', 'Stratum',
       'SubRound', 'SubSample', 'FOD_SubRegion', 'HG_SubBlkNo',
       'Second_Stratum', 'Visit_no', 'Hhold_no', 'Level', 'B3_q1', 'B3_q2',
       'B3_q3', 'B3_q4', 'HH_Type', 'B3_q5', 'B3_q6', 'B3_q7', 'B3_q8',
       'B3_q9', 'B3_q10', 'B3_q11', 'B3_q12', 'B3_q13', 'B3_q14', 'NSS', 'NSC',
       'MLT', 'Weight'],
      dtype='object')

In [300]:
# Keeping relevant variables and merging

# df1
df1 = df1[['HHID', 'State', 'B3_q13', 'Weight']].copy()        # MCE

# df2
df2 = df2[['HHID', 'B13_q1', 'B13_q3','B13_q4', 'B13_q7', 'B13_q8']].copy()     # Fin Assets
df2 = df2[df2['B13_q1'].isin([1,2,3,4,5,8,9,10,11])]

# df3
df3 = df3[['HHID', 'B12_q1', 'B12_q3', 'B12_q6']].copy()      # Shares
df3 = df3[df3['B12_q1'] == 8] 

# df4
df4 = df4[['HHID', 'B15_2_q1', 'B15_2_q6']].copy()         # Loan
df4 = df4[df4['B15_2_q6'].isin([1,2,3,4,5,6,7,8])]


# df5
df5 = df5[['HHID', 'B15_2_q1', 'B15_2_q23', 'B15_2_q24']].copy()     # Loan


# df6
df6 = df6[['HHID', 'B5_q1', 'B5_q5', 'B5_q11']].copy()                  # Land
df6 = df6[df6['B5_q1'] == 99]  



# df7
df7 = df7[['HHID', 'B6_q1', 'B6_q4']].copy()                      # Buildings
df7 = df7[df7['B6_q1'] == 11]  

# df8
df8 = df8[['HHID', 'B6_q1', 'B6_q14']].copy()                        # Buildings
df8 = df8[df8['B6_q1'] == 11]  

In [301]:
# Check for missing values in df1

cols = ['HHID', 'State', 'B3_q13', 'Weight']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [df1[col].isna().sum() for col in cols],
    'Zero_Values': [(df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

   Column  Missing_Values  Zero_Values
0    HHID               0            0
1   State               0            0
2  B3_q13               0            0
3  Weight               0            0
Unique HHIDs: 6342


In [302]:
df7.columns

Index(['HHID', 'B6_q1', 'B6_q4'], dtype='object')

In [303]:
# Renaming, creating, dropping columns 

df1  = df1.rename(columns={
    'B3_q13': 'MCE'
})

df2  = df2.rename(columns={
    'B13_q1': 'Fin_Asset_Serial',
    'B13_q3': 'Fin_Asset_val1_sign',
    'B13_q4': 'Fin_Asset_val1_absvalue',
    'B13_q7': 'Fin_Asset_val2_sign',
    'B13_q8': 'Fin_Asset_val2_absvalue'
})

df3  = df3.rename(columns={
    'B12_q1': 'Share_Asset_serial',
    'B12_q3': 'Share_Asset_val1',
    'B12_q6': 'Share_Asset_val2'
})

df4  = df4.rename(columns={
    'B15_2_q1': 'Liability_serial',
    'B15_2_q6': 'Credit_Agency'
})

df5  = df5.rename(columns={
    'B15_2_q1': 'Liability_serial',
    'B15_2_q23': 'Liability_val1',
    'B15_2_q24': 'Liability_val2'
})

df6  = df6.rename(columns={
    'B5_q1': 'land_serial',
    'B5_q5': 'land_val1',
    'B5_q11': 'land_val2'
})

df7  = df7.rename(columns={
    'B6_q1': 'Building_serial',
    'B6_q4': 'Building_val1',
})

df8  = df8.rename(columns={
    'B6_q1': 'Building_serial',
    'B6_q14': 'Building_val2',
})

# Creating
#df1['MCE'] = df1['MCE']/1000

In [304]:
df4.columns

Index(['HHID', 'Liability_serial', 'Credit_Agency'], dtype='object')

In [305]:
df5.columns

Index(['HHID', 'Liability_serial', 'Liability_val1', 'Liability_val2'], dtype='object')

In [306]:
# df2 (Financial Asset value)

# Alternatively, using `.where()` for better Pandas performance:
df2['Fin_Asset_val1'] = df2['Fin_Asset_val1_absvalue'].where(df2['Fin_Asset_val1_sign'] != 1, -df2['Fin_Asset_val1_absvalue'])
df2['Fin_Asset_val2'] = df2['Fin_Asset_val2_absvalue'].where(df2['Fin_Asset_val2_sign'] != 1, -df2['Fin_Asset_val2_absvalue'])


df2['Fin_Asset_Value'] = np.where(
    df2['Fin_Asset_Serial'] == 11,
    df2['Fin_Asset_val1'],
    df2['Fin_Asset_val1'].combine_first(df2['Fin_Asset_val2'])
)
df2['Fin_Asset_Value'] = pd.Series(df2['Fin_Asset_Value']).fillna(0)


# df3 (Share Asset value)

df3['Share_Asset_Value'] = df3['Share_Asset_val1'].combine_first(df3['Share_Asset_val2'])
df3['Share_Asset_Value'] = df3['Share_Asset_Value'].fillna(0)
df3 = df3.reset_index(drop=True)

# Share_Asset_Map

# Mapping dictionary using float keys (since your column is float64)
Share_Asset_Map = {
    8.0: 'Shares and Debentures'
}

# df6 (Land value)

df6['Land_value'] = df6['land_val1'].combine_first(df6['land_val2'])
df6['Land_value'] = df6['Land_value'].fillna(0)

In [307]:
df2['Fin_Asset_Serial'].value_counts()

Fin_Asset_Serial
11    6088
8     1842
4     1545
10    1211
3     1068
5      515
2      500
1      324
9       23
Name: count, dtype: int64

In [308]:
print(df2['Fin_Asset_Serial'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Fin_Asset_Value
summary_rows = df2.groupby('HHID', as_index=False)['Fin_Asset_Value'].sum()
summary_rows['Fin_Asset_Serial'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df2.columns:
    if col not in ['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value']:
        summary_rows[col] = np.nan

# Step 4: Concatenate the new rows
df2_mod = pd.concat([df2, summary_rows], ignore_index=True)

# Step 5: Sort
df2_mod = df2_mod.sort_values(by=['HHID', 'Fin_Asset_Serial']).reset_index(drop=True)

# Step 6: Save
df2_mod.to_stata("mergedforu.dta", write_index=False)

print(df2_mod.head())
print(df2_mod['HHID'].nunique())

df2_mod['Fin_Asset_Serial'].value_counts(dropna=False)

# Modify serials and labels, if any

# df2_mod
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([11]), 'Fin_Asset_Serial'] = 31
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([3,4,5]), 'Fin_Asset_Serial'] = 32
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([8]), 'Fin_Asset_Serial'] = 34
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([9,10]), 'Fin_Asset_Serial'] = 35
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([1,2]), 'Fin_Asset_Serial'] = 36
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([999]), 'Fin_Asset_Serial'] = 37

df2_mod = df2_mod.reset_index(drop=True)

# Fin Asset Mapping

Fin_Asset_map = {

    31.0: 'Currency',
    32.0: 'Deposits',
    34.0: 'Life Insurance fund',
    35.0: 'Provident and Pension fund',
    36.0: 'Claims on Government',
    37.0: 'Total financial assets (other thanshares and related instruments)'
}

df2_mod = df2_mod.groupby(['HHID', 'Fin_Asset_Serial'], as_index=False).agg({
    'Fin_Asset_Value': 'sum'  
})

df2_mod = df2_mod.reset_index(drop=True)

df2_mod.to_stata("mergedforu.dta", write_index=False)

df2_mod['Fin_Asset_Serial'].value_counts()

df2 = df2_mod.copy()

Fin_Asset_Serial
11    6088
8     1842
4     1545
10    1211
3     1068
5      515
2      500
1      324
9       23
Name: count, dtype: int64
         HHID  Fin_Asset_Serial  Fin_Asset_val1_sign  Fin_Asset_val1_absvalue  \
0  1100011102                11                  NaN                    400.0   
1  1100011102               999                  NaN                      NaN   
2  1100115102                 4                  NaN                    500.0   
3  1100115102                 8                  NaN                  36000.0   
4  1100115102                10                  NaN                  48000.0   

   Fin_Asset_val2_sign  Fin_Asset_val2_absvalue  Fin_Asset_val1  \
0                  NaN                      NaN           400.0   
1                  NaN                      NaN             NaN   
2                  NaN                    500.0           500.0   
3                  NaN                  34800.0         36000.0   
4                  NaN              

In [309]:
df4.head()

,HHID,Liability_serial,Credit_Agency
3,2409315101,1,2.0
5,2403015101,1,2.0
9,2490715102,1,3.0
10,2499025101,1,3.0
12,2497013101,1,6.0


In [310]:
# df4 and df5

# Merging both

# df4 and df5
mergedd_loan = pd.merge(df4, df5, on=['HHID', 'Liability_serial'], how='inner')

# Step 1: Compute Liability_value
mergedd_loan['Liability_value'] = mergedd_loan['Liability_val1'].combine_first(mergedd_loan['Liability_val2'])
mergedd_loan['Liability_value'] = mergedd_loan['Liability_value'].fillna(0)

# Step 2: Group by HHID and sum Liability_value
summary_rows_liab = mergedd_loan.groupby('HHID', as_index=False)['Liability_value'].sum()
summary_rows_liab['Credit_Agency'] = 999

# Step 3: Add other columns with NaN or defaults
for col in mergedd_loan.columns:
    if col not in ['HHID', 'Credit_Agency', 'Liability_value']:
        summary_rows_liab[col] = np.nan

# Step 4: Concatenate the new rows
merged_loan = pd.concat([mergedd_loan, summary_rows_liab], ignore_index=True)

# Step 5: Sort
merged_loan = merged_loan.sort_values(by=['HHID', 'Credit_Agency']).reset_index(drop=True)

# Step 6: Save
merged_loan.to_stata("mergedforu.dta", write_index=False)

print(merged_loan.head())
print(merged_loan['HHID'].nunique())

# merged_loan
merged_loan.loc[merged_loan['Credit_Agency'].isin([2,3]), 'Credit_Agency'] = 91
merged_loan.loc[merged_loan['Credit_Agency'].isin([1,4,5,6,7,8]), 'Credit_Agency'] = 92

merged_loan = merged_loan.reset_index(drop=True)

# Liability Map

Credit_Agency_Map = {

    91.0: 'Bank advances',
    92.0: 'Non-banking loans and advances',
    999.0: 'Total Institutional Liabilities'
}

merged_loan = merged_loan.groupby(['HHID', 'Credit_Agency'], as_index=False).agg({
    'Liability_value': 'sum'  
})

merged_loan = merged_loan.reset_index(drop=True)

merged_loan.to_stata("mergedforu_liab.dta", write_index=False)

merged_loan['Credit_Agency'].value_counts()

#df4 = merged_loan.copy()


         HHID  Liability_serial  Credit_Agency  Liability_val1  \
0  1100115102               3.0            2.0         65000.0   
1  1100115102               2.0            3.0         80400.0   
2  1100115102               4.0            3.0             NaN   
3  1100115102               5.0            3.0             NaN   
4  1100115102               1.0            5.0          6600.0   

   Liability_val2  Liability_value  
0         69000.0          65000.0  
1             NaN          80400.0  
2         20900.0          20900.0  
3         22350.0          22350.0  
4          9000.0           6600.0  
5435


Credit_Agency
999.0    5435
91.0     5040
92.0      683
Name: count, dtype: int64

In [311]:
# df7 and df8

# Outer merge df7 and df8 on 'HHID', 'Building_serial'

merged_building = pd.merge(df7, df8, on=['HHID', 'Building_serial'], how='outer')

merged_building['Building_value'] = merged_building['Building_val1'].combine_first(merged_building['Building_val2'])
merged_building['Building_value'] = merged_building['Building_value'].fillna(0)

merged_building.to_stata("mergedforu2.dta", write_index=False)

In [312]:
# Dropping unnecessary columns

merged_building.drop(['Building_serial', 'Building_val1', 'Building_val2'], axis=1, inplace=True)
#merged_loan.drop(['Liability_serial', 'Liability_val1','Liability_val2'], axis=1, inplace=True)
#df2.drop(['Fin_Asset_val1_sign',
#        'Fin_Asset_val1_absvalue', 'Fin_Asset_val2_sign',
#        'Fin_Asset_val2_absvalue', 'Fin_Asset_val1', 'Fin_Asset_val2'], axis=1, inplace=True)
df3.drop(['Share_Asset_val1', 'Share_Asset_val2'], axis=1, inplace=True)
df6.drop(['land_val1', 'land_val2', 'land_serial'], axis=1, inplace=True)


# Viewing before merging

df1.to_stata("df1_2003.dta", write_index=False)
df2.to_stata("df2_2003.dta", write_index=False)
df3.to_stata("df3_2003.dta", write_index=False)
df6.to_stata("df6_2003.dta", write_index=False)
merged_building.to_stata("merged_building_2003.dta", write_index=False)
merged_loan.to_stata("merged_loan_2003.dta", write_index=False)

In [313]:
df2.columns

Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value'], dtype='object')

In [314]:
# Groupby

df2 = df2.groupby(['HHID', 'Fin_Asset_Serial'], as_index=False).agg({
    'Fin_Asset_Value': 'sum'  
})

df3 = df3.groupby(['HHID', 'Share_Asset_serial'], as_index=False).agg({
    'Share_Asset_Value': 'sum'  
})

merged_loan = merged_loan.groupby(['HHID', 'Credit_Agency'], as_index=False).agg({
    'Liability_value': 'sum'  
})


In [315]:
dataframes['df1'] = df1
dataframes['df2'] = df2
dataframes['df3'] = df3
dataframes['df4'] = merged_loan
dataframes['df5'] = merged_building
dataframes['df6'] = df6


In [316]:
# Print unique HHIDs in first seven datasets
for i in range(1, 7):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 7)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 7):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6 = [dataframes[f'df{i}'] for i in range(1, 7)]

Unique HHIDs in df1: 6342
Unique HHIDs in df2: 6316
Unique HHIDs in df3: 6338
Unique HHIDs in df4: 5435
Unique HHIDs in df5: 6340
Unique HHIDs in df6: 6341
Common HHIDs across: 5406
df1: 5406 rows after filtering
df2: 16295 rows after filtering
df3: 5406 rows after filtering
df4: 11096 rows after filtering
df5: 5406 rows after filtering
df6: 5406 rows after filtering
df7: 13579 rows after filtering
df8: 13503 rows after filtering


In [317]:
# Merging all datasets
from functools import reduce

dfs = [df1, df2, df3, df4, df5, df6]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='HHID', how='outer'), dfs)

merged_df.to_stata("merged.dta", write_index=False)

unique_hhids = set(merged_df['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")


# Land value
#merged_df['Land_value'] = merged_df[['Urban_land_value', 'Rural_land_value']].sum(axis=1, min_count=1).fillna(0)

# Real Estate Value

merged_df['Real Estate'] = merged_df['Building_value'] + merged_df['Land_value']

merged_df.to_stata("merged1.dta", write_index=False)

merged_df1 = merged_df.copy()

merged_df1.columns

merged_df1.drop(['Building_value','Land_value'
                ], axis=1, inplace=True)

merged_df1.to_stata("merged1.dta", write_index=False)

merged_df1.columns

# Check for duplicates
group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']
dup_check = merged_df1.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

unique_count = merged_df1.loc[merged_df1['Fin_Asset_Serial'] == 37, 'HHID'].nunique()
print(f"Unique HHIDs with Fin_Asset_Serial = 37: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Share_Asset_serial'] == 8, 'HHID'].nunique()
print(f"Unique HHIDs with Share_Asset_serial = 8: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Credit_Agency'] == 999, 'HHID'].nunique()
print(f"Unique HHIDs with Credit_Agency = 999: {unique_count}")


Unique HHIDs: 5406
Number of duplicates: 0
Unique HHIDs with Fin_Asset_Serial = 37: 5406
Unique HHIDs with Share_Asset_serial = 8: 5406
Unique HHIDs with Credit_Agency = 999: 5406


C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\2437617522.py:21: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df.to_stata("merged1.dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\2437617522.py:30: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged1.dta", write_index=False)


In [318]:
cols = ['HHID', 'State','Weight', 'Fin_Asset_Serial', 'Share_Asset_serial',
        'Credit_Agency', 'Share_Asset_Value', 'Fin_Asset_Value',
        'Liability_value', 'Real Estate']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [merged_df1[col].isna().sum() for col in cols],
    'Zero_Values': [(merged_df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(merged_df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

merged_df1.to_stata("merged22.dta", write_index=False)


               Column  Missing_Values  Zero_Values
0                HHID               0            0
1               State               0            0
2              Weight               0            0
3    Fin_Asset_Serial               0            0
4  Share_Asset_serial               0            0
5       Credit_Agency               0            0
6   Share_Asset_Value               0           14
7     Fin_Asset_Value               0           19
8     Liability_value               0          329
9         Real Estate               0            0
Unique HHIDs: 5406


C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\3797293583.py:20: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged22.dta", write_index=False)


In [319]:
merged_df3 = merged_df1.copy()

# Create the new column
merged_df3['Fin_Asset'] = merged_df3['Fin_Asset_Serial'].map(Fin_Asset_map)
merged_df3['Share_Asset'] = merged_df3['Share_Asset_serial'].map(Share_Asset_Map)
merged_df3['Liability'] = merged_df3['Credit_Agency'].map(Credit_Agency_Map)

# HHIDs satisfying each condition
hhid_fin = set(merged_df3.loc[merged_df3['Fin_Asset_Serial'] == 37, 'HHID'])
hhid_share = set(merged_df3.loc[merged_df3['Share_Asset_serial'] == 8, 'HHID'])
hhid_credit = set(merged_df3.loc[merged_df3['Credit_Agency'] == 999, 'HHID'])

# Intersection: HHIDs present in all three
common_hhids = hhid_fin & hhid_share & hhid_credit

# Keep only those rows
filtered_df = merged_df3[merged_df3['HHID'].isin(common_hhids)].copy()

# Print how many unique HHIDs remain
print(f"Unique HHIDs present in all three groups: {len(common_hhids)}")

#filtered_df.drop(['Fin_Asset', 'Share_Asset', 'Liability'], axis=1, inplace=True)

filtered_df.to_stata("final(1)_13.dta", write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\2063279316.py:24: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  filtered_df.to_stata("final(1)_13.dta", write_index=False)


Unique HHIDs present in all three groups: 5406


In [320]:
# Sanity checks if grouping worked correctly

print(sorted(filtered_df['Fin_Asset_Serial'].unique()))
#print(filtered_df['Fin_Asset_Serial'].value_counts())

print(sorted(filtered_df['Share_Asset_serial'].unique()))
#print(filtered_df['Share_Asset_serial'].value_counts())

print(sorted(filtered_df['Credit_Agency'].unique()))
#print(filtered_df['Credit_Agency'].value_counts())

[31, 32, 34, 35, 36, 37]
[8]
[91.0, 92.0, 999.0]


In [321]:
df_grouped1 = filtered_df.copy()

# Step 1: Filter the relevant rows satisfying your condition
wealth_rows = df_grouped1[
    (df_grouped1['Credit_Agency'] == 999) &
    (df_grouped1['Fin_Asset_Serial'] == 37) &
    (df_grouped1['Share_Asset_serial'] == 8)
].copy()

# Step 2: Compute wealth for these rows
wealth_rows['Gross wealth'] = (
    wealth_rows['Fin_Asset_Value'] +
    wealth_rows['Share_Asset_Value'] +
    wealth_rows['Real Estate']
)

wealth_rows['Total Financial Assets'] = (
    wealth_rows['Fin_Asset_Value']+
    wealth_rows['Share_Asset_Value']
)

wealth_rows['Total Institutional liabilities'] = wealth_rows['Liability_value']

# Step 3: Keep only HHID and wealth columns
wealth_per_hhid = wealth_rows[['HHID', 'Gross wealth', 'Total Financial Assets', 'Total Institutional liabilities']]

# Step 4: Merge wealth back to the full dataset based on HHID
df_grouped1 = df_grouped1.merge(wealth_per_hhid, on='HHID', how='left')

# Step 5: View result
print(df_grouped1[['HHID', 'Gross wealth']].drop_duplicates().head())

df_grouped1.to_stata("final(25).dta", write_index=False)



          HHID  Gross wealth
0   1100115102      554900.0
15  1100316101      262341.0
21  1100321101      196542.0
31  1100411102      180566.0
37  1100412101      430280.0


C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\3773941852.py:33: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped1.to_stata("final(25).dta", write_index=False)


In [322]:
df_grouped1.columns

Index(['HHID', 'State', 'MCE', 'Weight', 'Fin_Asset_Serial', 'Fin_Asset_Value',
       'Share_Asset_serial', 'Share_Asset_Value', 'Credit_Agency',
       'Liability_value', 'Real Estate', 'Fin_Asset', 'Share_Asset',
       'Liability', 'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities'],
      dtype='object')

In [323]:
df_grouped2 =  df_grouped1.copy()

import numpy as np

# Initialize columns with NaN
df_grouped2['Asset_serial'] = np.nan
df_grouped2['Asset_Value'] = np.nan
df_grouped2['Asset_Name'] = np.nan

group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']

df_grouped2['HHID'] = df_grouped2['HHID'].astype(str)

#df_grouped2.to_stata("final(26).dta", write_index=False)


df_grouped2.to_stata("final(26).dta", write_index=False)


# Empty list to store new rows
rows = []

# Loop over each unique group
for _, group in df_grouped2.groupby(group_cols):

    # Get first row for copying other columns
    first_row = group.iloc[0].to_dict()

    # 1. Financial Asset Row
    if not np.isnan(first_row['Fin_Asset_Value']):
        fa_row = first_row.copy()
        fa_row['Asset_serial'] = first_row['Fin_Asset_Serial']
        fa_row['Asset_Value'] = first_row['Fin_Asset_Value']
        fa_row['Asset_Name'] = first_row['Fin_Asset']
        rows.append(fa_row)

    # 2. Share Asset Row
    if not np.isnan(first_row['Share_Asset_Value']):
        sa_row = first_row.copy()
        sa_row['Asset_serial'] = 200 + first_row['Share_Asset_serial']  # As you specified
        sa_row['Asset_Value'] = first_row['Share_Asset_Value']
        sa_row['Asset_Name'] = first_row['Share_Asset']
        rows.append(sa_row)

    # 3. Real Estate Row
    if not np.isnan(first_row['Real Estate']):
        re_row = first_row.copy()
        re_row['Asset_serial'] = 300
        re_row['Asset_Value'] = first_row['Real Estate']
        re_row['Asset_Name'] = 'Real Estate'
        rows.append(re_row)

    # 4. Gross wealth Row
    if not np.isnan(first_row['Gross wealth']):
        w_row = first_row.copy()
        w_row['Asset_serial'] = 1000
        w_row['Asset_Value'] = first_row['Gross wealth']
        w_row['Asset_Name'] = 'Gross wealth'
        rows.append(w_row)

    # 6. Total Financial Assets Row
    if not np.isnan(first_row['Total Financial Assets']):
        tfa_row = first_row.copy()
        tfa_row['Asset_serial'] = 1500
        tfa_row['Asset_Value'] = first_row['Total Financial Assets']
        tfa_row['Asset_Name'] = 'Total Financial Assets'
        rows.append(tfa_row)   

# Convert to DataFrame
asset_df = pd.DataFrame(rows)

# Optional: Arrange columns nicely
cols_order = group_cols + ['Asset_serial', 'Asset_Value'] + [col for col in df_grouped2.columns if col not in group_cols + ['Asset_serial', 'Asset_Value']]
asset_df = asset_df[cols_order]

# Check result
print(asset_df.head())



# Renaming Liabilities

asset_df = asset_df.rename(columns={
    'Credit_Agency': 'Liability_serial',
    'Liability': 'Liability_name'
})

asset_df.to_stata("final(28).dta", write_index=False)



C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\1759378772.py:17: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped2.to_stata("final(26).dta", write_index=False)


         HHID  Credit_Agency  Fin_Asset_Serial  Share_Asset_serial  \
0  1100115102           91.0                31                   8   
1  1100115102           91.0                31                   8   
2  1100115102           91.0                31                   8   
3  1100115102           91.0                31                   8   
4  1100115102           91.0                31                   8   

   Asset_serial  Asset_Value  State     MCE   Weight  Fin_Asset_Value  \
0            31        400.0     33  4500.0  3.85125            400.0   
1           208      20000.0     33  4500.0  3.85125            400.0   
2           300     450000.0     33  4500.0  3.85125            400.0   
3          1000     554900.0     33  4500.0  3.85125            400.0   
4          1500     104900.0     33  4500.0  3.85125            400.0   

   Share_Asset_Value  Liability_value  Real Estate Fin_Asset  \
0            20000.0         188650.0     450000.0  Currency   
1           

C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\1759378772.py:88: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  asset_df.to_stata("final(28).dta", write_index=False)


In [324]:
group_cols = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial']
group_cols1 = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial', 'Asset_serial']

# Count unique combinations
num_subsets = asset_df[group_cols].drop_duplicates().shape[0]

print(f"Number of unique subsets: {num_subsets}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols1)
print(f"Number of duplicates: {dup_check.sum()}")

Number of unique subsets: 33670
Number of duplicates: 134680
Number of duplicates: 0


In [325]:
# Check for missing values

# Check how many missing in each column
print(asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().sum())

# Optionally, view rows with missing in any of them
missing_rows = asset_df[
    asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().any(axis=1)
]

print(f"\nTotal rows with missing values: {len(missing_rows)}")
print(missing_rows.head())


Asset_serial    0
Asset_Value     0
Asset_Name      0
dtype: int64

Total rows with missing values: 0
Empty DataFrame
Columns: [HHID, Liability_serial, Fin_Asset_Serial, Share_Asset_serial, Asset_serial, Asset_Value, State, MCE, Weight, Fin_Asset_Value, Share_Asset_Value, Liability_value, Real Estate, Fin_Asset, Share_Asset, Liability_name, Gross wealth, Total Financial Assets, Total Institutional liabilities, Asset_Name]
Index: []


In [326]:
final_df = asset_df.copy()

final_df.drop(['Fin_Asset_Serial', 
               'Share_Asset_serial', 
               'Fin_Asset_Value',
                'Share_Asset_Value', 
                'Fin_Asset',
                'Share_Asset',               
                ], axis=1, inplace=True)

final_df.to_stata("final(2013).dta", write_index=False)

final_df = final_df[['HHID', 'State' ,'MCE', 'Weight','Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value',
                    'Real Estate', 'Total Financial Assets', 'Total Institutional liabilities', 
                    ]].copy()

final_df.to_stata("final(2013).dta", write_index=False)

final_df['HHID'].nunique()

final_df1 = final_df[['HHID', 'State', 'MCE', 'Weight', 'Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value'
                    ]].copy()

final_df1.to_stata("final(2013)_1.dta", write_index=False)
final_df1['HHID'].nunique()

C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\3734631850.py:11: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df.to_stata("final(2013).dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\3734631850.py:19: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth
    Real Estate   ->   Real_Estate
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional

5406

In [327]:
group_cols1 = ['HHID', 'Liability_serial', 'Asset_serial']

final_df2 = final_df1.drop_duplicates(subset=group_cols1, keep='first').reset_index(drop=True)

# Check for repetition

# Check for uniqueness 

duplicates = final_df2.duplicated(subset=['Asset_serial', 'HHID', 'Liability_serial'], keep=False)

# Check if there are any duplicates
if duplicates.any():
    print("Some (Asset_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Asset_serial', 'HHID']].sort_values(['Asset_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Asset_serial category.")

# Check if there are any duplicates
if duplicates.any():
    print("Some (Liability_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Liability_serial', 'HHID']].sort_values(['Liability_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Liability_serial category.")


 All HHIDs are unique within each Asset_serial category.
 All HHIDs are unique within each Liability_serial category.


In [328]:
final_df2['HHID'].nunique()

5406

# Add State names corresponding to their codes

In [329]:
# final_df2 (Merge D&D, D&N)
final_df2.loc[final_df2['State'].isin([25,26]), 'State'] = 25
final_df2['State'].value_counts()

# State Map

State_map = {
    1.0: 'Jammu and Kashmir',
    2.0: 'Himachal Pradesh',
    3.0: 'Punjab',
    4.0: 'Chandigarh',
    5.0: 'Uttarakhand',
    6.0: 'Haryana',
    7.0: 'Delhi',
    8.0: 'Rajasthan',
    9.0: 'Uttar Pradesh',
    10.0: 'Bihar',
    11.0: 'Sikkim',
    12.0: 'Arunachal Pradesh',
    13.0: 'Nagaland',
    14.0: 'Manipur',
    15.0: 'Mizoram',
    16.0: 'Tripura',
    17.0: 'Meghalaya',
    18.0: 'Assam',
    19.0: 'West Bengal',
    20.0: 'Jharkhand',
    21.0: 'Odisha',
    22.0: 'Chhattishgarh',
    23.0: 'Madhya Pradesh',
    24.0: 'Gujarat',
    25.0: 'Daman and Diu and Dadra and Nagar Haveli',
    27.0: 'Maharashtra',
    28.0: 'Andhra Pradesh',
    29.0: 'Karnataka',
    30.0: 'Goa',
    31.0: 'Lakshadweep',
    32.0: 'Kerala',
    33.0: 'Tamilnadu',
    34.0: 'Puducherry',
    35.0: 'Andaman & Nicobar'
}

final_df3 = final_df2.copy()

final_df3['State_name'] = final_df3['State'].map(State_map)

final_df3.to_stata('2003_final.dta', write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_1412\1591137529.py:48: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df3.to_stata('2003_final.dta', write_index=False)
